In [1]:
'''
ryu-lab-06-2-softmax_zoo_classifier-DNN 를 수정해서 IVR서비스 추천을 위한 Collaborative Filtering 으로 사용
2017.10.9
류기동
'''
import tensorflow as tf
import numpy as np
import datetime

In [2]:

tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
#xy = np.loadtxt('./IVRService/CfDataSmall-20171009.csv', delimiter=',', dtype=np.float32)
xy = np.loadtxt('./IVRService/CfData-Y01-20171015.csv', delimiter=',', dtype=np.float32)
'''
CfDataSmall 은 전체 중에 1006개의 레코드만 가진 샘플 데이터
UserID, ServiceCODE, 사용여부 3개의 컬럼 가지고 있음
Y=0 인것의 갯수 7932, Y=1 인것의 갯수 7932
record수는 15863 개
2,100900,0
2,100901,0
2,100902,0
2,100903,1
2,100904,0

'''

'\nCfDataSmall 은 전체 중에 1006개의 레코드만 가진 샘플 데이터\nUserID, ServiceCODE, 사용여부 3개의 컬럼 가지고 있음\nY=0 인것의 갯수 7932, Y=1 인것의 갯수 7932\nrecord수는 15863 개\n2,100900,0\n2,100901,0\n2,100902,0\n2,100903,1\n2,100904,0\n\n'

In [3]:
x_data = xy[:, 0:-1]  #UserID, Service Code
y_data = xy[:, [-1]]  #서비스 사용여부

print(x_data.shape, y_data.shape)

print (x_data.shape[1])

x1_data = x_data[:, [0]]  #UserID부분  => Embedding Layer의 입력
x2_data = x_data[:, [1]]  #Service Code 부분 => Embedding Layer의 입력

(15864, 2) (15864, 1)
2


In [4]:
# one-hot 으로 데이터 변환
#X1은 User ID : 234가지 종류, X2는 Service Code,  43가지
x_col = x_data.shape[1]
y_col = y_data.shape[1]
print (x_col, y_col)


2 1


In [5]:

#X = tf.placeholder(tf.float32, [None, x_col])  #이거는 필요 없을듯
X1 = tf.placeholder(tf.int32, [None, 1])  #one hot 을 할려면 int32로 해야 한다. float32는 안된다.
X2 = tf.placeholder(tf.int32, [None, 1])
Y = tf.placeholder(tf.int32, [None, y_col])  # 


In [6]:
#Y_one_hot = tf.one_hot(Y, 7)

In [7]:
#one-hot 으로 변경 하기
#X1_classes = 234
X1_classes = 2495
X2_classes = 43
Y_OUT = 2
X1_one_hot = tf.one_hot(X1, X1_classes)  # one hot
print("X1 one_hot", X1_one_hot)
X1_one_hot = tf.reshape(X1_one_hot, [-1, X1_classes])
print("X1 reshape", X1_one_hot)
X2_one_hot = tf.one_hot(X2, X2_classes)  # one hot
print("X2 one_hot", X2_one_hot)
X2_one_hot = tf.reshape(X2_one_hot, [-1, X2_classes])
print("X2 reshape", X2_one_hot)

Y_one_hot = tf.one_hot(Y, Y_OUT)  # one hot
print("Y one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, Y_OUT])
print("Y reshape", Y_one_hot)



X1 one_hot Tensor("one_hot:0", shape=(?, 1, 2495), dtype=float32)
X1 reshape Tensor("Reshape:0", shape=(?, 2495), dtype=float32)
X2 one_hot Tensor("one_hot_1:0", shape=(?, 1, 43), dtype=float32)
X2 reshape Tensor("Reshape_1:0", shape=(?, 43), dtype=float32)
Y one_hot Tensor("one_hot_2:0", shape=(?, 1, 2), dtype=float32)
Y reshape Tensor("Reshape_2:0", shape=(?, 2), dtype=float32)


In [8]:
EL_OUT_1 = 256
EL_OUT_2 = 16
HIDDEN_OUT_1 = 128
HIDDEN_OUT_2 = 64
HIDDEN_OUT_3 = 32
HIDDEN_OUT_4 = 32


# dropout (keep_prob) rate  0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

#Embedding Layer EL1 은 X1의 Embedding, EL2는 X2의 Embedding
#Embedding을 하고 나면 차원이 줄어든다. 
# EL1 = X1*WE1 + bE1
# EL2 = X2*WE2 + bE2
#EL1, EL2가 실질적인 INPUT 값 X의 역할을 한다고 보면 된다., 겁나큰 스파스한 벡터를 덴스하게 바꿔서 MLP에 
#입력으로 사용한다는 의미

#Layer1(첫번째) 에서는 EL1, EL2를 합쳐서 받아야 하므로 
# L1 = (EL1 + EL2) * W1 + b1 이 된다. 
# L2 = L1 * W2 + b2
#....
'''
WEL1  이 어쨌든 첫번째 입력 단이다. 여기서 X1은 x1_data 가 feed 된다. x1_data 는 X1_one_hot으로 변경된다.
X1_one_hot은 ?, 234 이므로 WEL1(EL1의 Weight) 의 shape는 234, EL_OUT_1(32) 가 된다. 즉 234차원을 32차원으로 
변형(임베딩) 하는 것이다.

'''
with tf.name_scope("Embedding_Layer_User") as scope :
    WEL1 = tf.get_variable("WEL1", shape=[X1_classes, EL_OUT_1], initializer=tf.contrib.layers.xavier_initializer() )
    bE1 = tf.Variable(tf.random_normal([EL_OUT_1]))
    LE1 = tf.nn.relu(tf.matmul(X1_one_hot, WEL1) + bE1)

with tf.name_scope("Embedding_Layer_Service") as scope :
    WEL2 = tf.get_variable("WEL2", shape=[X2_classes, EL_OUT_2], initializer=tf.contrib.layers.xavier_initializer() )
    bE2 = tf.Variable(tf.random_normal([EL_OUT_2]))
    LE2 = tf.nn.relu(tf.matmul(X2_one_hot, WEL2) + bE2)



In [9]:
'''
L1은 EL1, EL2가 합쳐지는 계층, MLP에서는 첫번째 Hidden 이 된다. 
입력은 EL1 + EL2 크기이고 출력은 HIDDEN_OUT_1 이 된다.
입력 값은 LE1 과 LE2를 tf.contact 해서 행렬을 맞춘다.
'''
with tf.name_scope("Input_Layer") as scope :
    L1_in = tf.concat([LE1, LE2],1)
    print(L1_in.shape) # 32+ 16 = 48
    print(LE1.shape)

(?, 272)
(?, 256)


In [10]:
#L1_in.shape
with tf.name_scope("Layer1") as scope :
    W1 = tf.get_variable("W1", shape=[EL_OUT_1+EL_OUT_2, HIDDEN_OUT_1], initializer=tf.contrib.layers.xavier_initializer() )
    b1 = tf.Variable(tf.random_normal([HIDDEN_OUT_1]))
    L1 = tf.nn.relu(tf.matmul(L1_in, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


In [11]:
with tf.name_scope("Layer2") as scope :
    W2 = tf.get_variable("W2", shape=[HIDDEN_OUT_1,HIDDEN_OUT_2], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([HIDDEN_OUT_2]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
with tf.name_scope("Layer3") as scope :
    W3= tf.get_variable("W3", shape=[HIDDEN_OUT_2,HIDDEN_OUT_3], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([HIDDEN_OUT_3]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
with tf.name_scope("Layer4") as scope :
    W4= tf.get_variable("W4", shape=[HIDDEN_OUT_3,HIDDEN_OUT_4], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([HIDDEN_OUT_4]))
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

In [12]:
#Last Layer (FC) and Hypothesis
#variable 이름이 들어가는 코드는 컴파일을 다시 할 수 없다. 
with tf.name_scope("FC_Layer") as scope :
    WFC = tf.get_variable("WFC", shape=[HIDDEN_OUT_4, Y_OUT],
                         initializer=tf.contrib.layers.xavier_initializer())
    bFC = tf.Variable(tf.random_normal([Y_OUT]))
    logit = tf.matmul(L4, WFC) + bFC
    hypothesis = tf.nn.softmax(logit)
#hypothesis = tf.matmul(L3, WFC) + bFC
#hypothesis = tf.sigmoid(tf.matmul(L3, WFC) + bFC)

In [13]:
# parameters
learning_rate = 0.05
training_epochs = 300
batch_size = 1000 # batch size는 크면 학습량이 작다

#logit = tf.matmul(L3, WFC) + bFC
#hypothesis = tf.nn.softmax(logit)
#define cost/loss & optimizer
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#softmax 는 logit을 입력으로 받는다. logit은 0~1까지 확률값을 가진값이다.
#예을 들어 y_one_hot 이 2개 클래스에 대한 값이면 logit은 각각에 대해 0~1까지 확률 값이 나오고
#softmax는 그값을 받아서 2개 클래스 중에 한개 값이 많이 크게 만들어 준다.
with tf.name_scope("Cost") as scope :
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=Y_one_hot))
    cost_str = tf.summary.scalar("Cost", cost)
#cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
#                       tf.log(1 - hypothesis))
with tf.name_scope("Train") as scope :
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [14]:
x1_data.shape[0]

15864

In [15]:
print('[%s]'% datetime.datetime.now(), 'Epoch:', '%04d' % (1))

[2017-10-24 01:08:32.266660] Epoch: 0001


In [16]:
#학슴 하면서 테스트 할 것 들 있으면 계산식을 미리 선언한다.
prediction = tf.argmax(hypothesis, 1)
correction_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary = tf.summary.merge_all()
#merged = tf.merge_all_summaries()
writer = tf.summary.FileWriter("./logs/test", sess.graph )
#writer = tf.summary.FileWriter("./logs/test", sess.graph_def )
#writer.add_graph(sess.graph)

n_deg = 1
# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    avg_acc = 0
    total_batch = int(x1_data.shape[0] / batch_size)  #10062/batch_size
    #total_batch = int(x1_data.shape[0]) # 101개 밖에 안되니까 그냥 다 돌린다.
    
    ############################
    #Mini batch 구간
    for i in range(total_batch):  
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        '''
        if i % 50  == 0 : 
            loss, acc = sess.run([cost, accuracy], feed_dict= {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 1})
            print('BatchRun:', '%03d'%(i+1), 'cost=', '{:.9f}'.format(loss),
                 'acc=', '{:.9f}'.format(acc))
        '''
        #if i % 25  == 0 : print('BatchRun:', '%03d'%(i+1), 'cost=', '{:.9f}'.format(avg_cost))
      # datetime.datetime.now() 
    
    
    if epoch % n_deg == 0 : 
        print('[%s]'% datetime.datetime.now(), 'Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
        #summary_str = sess.run(summary, feed_dict = {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 1})
        #writer.add_summary(summary_str, epoch)

print('Learning Finished!')



[2017-10-24 01:09:13.703729] Epoch: 0001 cost = 4.006233251
[2017-10-24 01:09:52.080126] Epoch: 0002 cost = 0.712026350
[2017-10-24 01:10:32.126624] Epoch: 0003 cost = 0.695269632
[2017-10-24 01:11:12.386834] Epoch: 0004 cost = 0.693684812
[2017-10-24 01:11:50.575537] Epoch: 0005 cost = 0.693231988
[2017-10-24 01:12:30.773635] Epoch: 0006 cost = 0.693093673
[2017-10-24 01:13:08.789675] Epoch: 0007 cost = 0.692700422
[2017-10-24 01:13:46.873710] Epoch: 0008 cost = 0.692252103
[2017-10-24 01:14:25.475788] Epoch: 0009 cost = 0.692409066
[2017-10-24 01:15:03.529431] Epoch: 0010 cost = 0.691801838
[2017-10-24 01:15:43.355685] Epoch: 0011 cost = 0.691658207
[2017-10-24 01:16:24.126611] Epoch: 0012 cost = 0.691814439
[2017-10-24 01:17:02.175602] Epoch: 0013 cost = 0.691790744
[2017-10-24 01:17:42.452676] Epoch: 0014 cost = 0.691726581
[2017-10-24 01:18:21.120606] Epoch: 0015 cost = 0.691534126
[2017-10-24 01:18:59.149288] Epoch: 0016 cost = 0.691673752
[2017-10-24 01:19:37.343400] Epoch: 0017

[2017-10-24 02:37:48.841662] Epoch: 0138 cost = 0.690135992
[2017-10-24 02:38:27.000732] Epoch: 0139 cost = 0.690125247
[2017-10-24 02:39:07.201894] Epoch: 0140 cost = 0.690110556
[2017-10-24 02:39:46.229580] Epoch: 0141 cost = 0.690230783
[2017-10-24 02:40:26.170915] Epoch: 0142 cost = 0.690144463
[2017-10-24 02:41:04.447068] Epoch: 0143 cost = 0.690175172
[2017-10-24 02:41:43.031454] Epoch: 0144 cost = 0.690146486
[2017-10-24 02:42:21.285619] Epoch: 0145 cost = 0.690085888
[2017-10-24 02:42:59.521775] Epoch: 0146 cost = 0.690243908
[2017-10-24 02:43:37.684839] Epoch: 0147 cost = 0.690104294
[2017-10-24 02:44:17.748910] Epoch: 0148 cost = 0.690217113
[2017-10-24 02:44:56.378861] Epoch: 0149 cost = 0.690090322
[2017-10-24 02:45:36.224444] Epoch: 0150 cost = 0.690054866
[2017-10-24 02:46:14.135514] Epoch: 0151 cost = 0.690169624
[2017-10-24 02:46:52.105241] Epoch: 0152 cost = 0.690208670
[2017-10-24 02:47:30.345528] Epoch: 0153 cost = 0.690161788
[2017-10-24 02:48:08.520756] Epoch: 0154

[2017-10-24 04:05:39.473956] Epoch: 0275 cost = 0.691210322
[2017-10-24 04:06:16.762988] Epoch: 0276 cost = 0.691177793
[2017-10-24 04:06:56.515922] Epoch: 0277 cost = 0.691151857
[2017-10-24 04:07:34.619490] Epoch: 0278 cost = 0.691178878
[2017-10-24 04:08:14.279281] Epoch: 0279 cost = 0.691146000
[2017-10-24 04:08:51.598318] Epoch: 0280 cost = 0.691152159
[2017-10-24 04:09:29.065528] Epoch: 0281 cost = 0.691159050
[2017-10-24 04:10:06.478361] Epoch: 0282 cost = 0.691168785
[2017-10-24 04:10:43.828060] Epoch: 0283 cost = 0.691187286
[2017-10-24 04:11:21.080546] Epoch: 0284 cost = 0.691172008
[2017-10-24 04:12:00.315076] Epoch: 0285 cost = 0.691185554
[2017-10-24 04:12:38.274721] Epoch: 0286 cost = 0.691141748
[2017-10-24 04:13:17.745676] Epoch: 0287 cost = 0.691145631
[2017-10-24 04:13:55.210037] Epoch: 0288 cost = 0.691153463
[2017-10-24 04:14:32.441689] Epoch: 0289 cost = 0.691181962
[2017-10-24 04:15:09.762420] Epoch: 0290 cost = 0.691190128
[2017-10-24 04:15:47.278387] Epoch: 0291

In [17]:
     loss, acc = sess.run([cost, accuracy], feed_dict= {X1: x1_data, X2: x2_data,  Y: y_data, keep_prob: 1})
     print('BatchRun:', '%03d'%(i+1), 'cost=', '{:.9f}'.format(loss),
                 'acc=', '{:.9f}'.format(acc))

BatchRun: 015 cost= 0.691088140 acc= 0.502962708


In [18]:
# Let's see if we can predict
prediction = tf.argmax(hypotesis, 1)
correction_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
pred = sess.run(prediction, feed_dict={X1: x1_data,X2:x2_data, keep_prob: 1})
# y_data: (N,1) = flatten => (N, ) matches pred.shape
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


NameError: name 'hypotesis' is not defined

In [19]:
# Let's see if we can predict
#prediction = tf.cast(hypothesis > 0.5, dtype=tf.int32)
prediction = hypothesis
pred = sess.run(prediction, feed_dict={X1: x1_data,X2:x2_data, keep_prob: 1})


In [20]:
# y_data: (N,1) = flatten => (N, ) matches pred.shape
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

...

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 0
[[False False]] Prediction: [ 0.50142664  0.49857333] True Y: 1
[[False False]] Prediction: [ 0.50142664

In [ ]:
print(pred)